In [16]:
import rosbag
from cv_bridge import CvBridge
import cv2
import os
import numpy as np
import csv

In [2]:
bag_paths = ['bags/' + i for i in os.listdir('bags/')]
bag_paths.sort()

In [3]:
bridge = CvBridge()
# ITERATE OVER EACH BAG
for bag_path in bag_paths:
    print(f"using bag file {bag_path}")
    bag = rosbag.Bag(bag_path, 'r')

    # READ ALL TOPICS FROM THE BAG
    topics = bag.get_type_and_topic_info()[1].keys()
    topics = list(topics)

    # GET LIST OF ALL IMAGE TOPICS IN THE BAG
    imageTopics = []
    for t in topics:
        topicTuple = bag.get_type_and_topic_info()[1][t]
        if topicTuple.msg_type == 'sensor_msgs/Image':
            imageTopics.append(t)
    break
    # GET MESSAGES FOR A SPECIFIC TOPIC WITH IMAGE MESSAGES
    for topic in imageTopics:
        print(f"\t writing for topic {topic}")        
        dirName = 'imgs/' + topic.replace('/','_')

        for _, msg, t in bag.read_messages(topic):
            
            cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding=msg.encoding)
            timestamp = str(t.secs) + str(t.nsecs)
            fname = dirName + "/" + timestamp + '.png'
            
            if os.path.exists(fname):
                print(f"{fname} already exists in the bag")
            cv2.imwrite(fname, cv_image)

using bag file bags/2023-03-14-12-16-12_37b.bag


In [6]:
imu_topic_compensated = '/imu/imu_compensated'
imu_topic_uncompensated = '/imu/imu_uncompensated'

In [7]:
for _, msg, t in bag.read_messages(imu_topic_compensated):
    print(msg)
    break

header: 
  seq: 273168
  stamp: 
    secs: 1678810589
    nsecs: 997018388
  frame_id: "imu"
orientation: 
  x: -0.0016885442892089486
  y: 0.0018819085089489818
  z: 0.9666464924812317
  w: -0.256102055311203
orientation_covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
angular_velocity: 
  x: -0.02170119248330593
  y: -0.00548772606998682
  z: 0.02348034642636776
angular_velocity_covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
linear_acceleration: 
  x: -1.8400936126708984
  y: -0.03929905593395233
  z: -9.802037239074707
linear_acceleration_covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [22]:
header = [
    'time',
    'orientation_x',
    'orientation_y',
    'orientation_z',
    'orientation_w',
    'angular_velocity_x',
    'angular_velocity_y',
    'angular_velocity_z',
    'linear_acceleration_x',
    'linear_acceleration_y',
    'linear_acceleration_z',
]

csv_filename = "imu_data_compensated.csv"

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  # Write header

csv_filename_uncompensated = "imu_data_uncompensated.csv"

with open(csv_filename_uncompensated, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  # Write header

In [23]:
for bag_path in bag_paths:
    print(f"Using bag file {bag_path}")
    bag = rosbag.Bag(bag_path, 'r')
    
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        i = 0
        for _, msg, t in bag.read_messages(imu_topic_compensated):
            row = [
                    msg.header.stamp.to_nsec(),
                    msg.orientation.x,
                    msg.orientation.y,
                    msg.orientation.z,
                    msg.orientation.w,
                    msg.angular_velocity.x,
                    msg.angular_velocity.y,
                    msg.angular_velocity.z,
                    msg.linear_acceleration.x,
                    msg.linear_acceleration.y,
                    msg.linear_acceleration.z,
                ]
            writer.writerow(row)  # Write data
            i+= 1
        print(f"Wrote {i} messages for compensated imu")

    with open(csv_filename_uncompensated, mode='a', newline='') as file:
        writer = csv.writer(file)
        i = 0
        for _, msg, t in bag.read_messages(imu_topic_uncompensated):
            row = [
                    msg.header.stamp.to_nsec(),
                    msg.orientation.x,
                    msg.orientation.y,
                    msg.orientation.z,
                    msg.orientation.w,
                    msg.angular_velocity.x,
                    msg.angular_velocity.y,
                    msg.angular_velocity.z,
                    msg.linear_acceleration.x,
                    msg.linear_acceleration.y,
                    msg.linear_acceleration.z,
                ]
            writer.writerow(row)  # Write data
            i+= 1
        print(f"Wrote {i} messages for uncompensated imu")

Using bag file bags/2023-03-14-12-16-12_37b.bag
Wrote 2964 messages for compensated imu
Wrote 2964 messages for uncompensated imu
Using bag file bags/2023-03-14-12-16-44_38.bag
Wrote 6374 messages for compensated imu
Wrote 6374 messages for uncompensated imu
Using bag file bags/2023-03-14-12-17-16_39.bag
Wrote 6362 messages for compensated imu
Wrote 6363 messages for uncompensated imu
Using bag file bags/2023-03-14-12-17-48_40.bag
Wrote 6372 messages for compensated imu
Wrote 6371 messages for uncompensated imu
Using bag file bags/2023-03-14-12-18-20_41.bag
Wrote 6373 messages for compensated imu
Wrote 6373 messages for uncompensated imu
Using bag file bags/2023-03-14-12-18-52_42.bag
Wrote 6373 messages for compensated imu
Wrote 6373 messages for uncompensated imu
Using bag file bags/2023-03-14-12-19-24_43.bag
Wrote 6363 messages for compensated imu
Wrote 6364 messages for uncompensated imu
Using bag file bags/2023-03-14-12-19-55_44.bag
Wrote 6372 messages for compensated imu
Wrote 637

In [ ]:
for msg in 